In [18]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import tensorflow as tf

### Load validation set in tensorflow format

In [19]:
def load_dataset(path):
    data = pd.read_pickle(path)
    n_features = data['clip_features'][0].shape[-1]
    class_names = data['y'].unique()
    y = np.array([np.where(class_names == e)[0][0] for e in data['y']])
    x = np.concatenate(data['clip_features'].to_numpy())
    print(f'Loaded {len(x)} instances from {path}.')
    print(f'X shape {x.shape} and y shape {y.shape} with labels:\n{data["y"].value_counts()}.')
    train_ds = tf.data.Dataset.from_tensor_slices((x, y)).batch(16)
    return train_ds, class_names, n_features, data['paths']


validation_ds, class_names, n_features, paths = load_dataset('/home/theo/robot_images/validation.pk')

Loaded 22480 instances from /home/theo/robot_images/validation.pk.
X shape (22480, 512) and y shape (22480,) with labels:
y
class_Superior human                     22464
class_Thinking machine                       4
class_None of the above                      2
class_Mysterious AI                          2
class_Collaborative or Interactive AI        2
class_Complex AI                             2
class_Learning or recognition machine        2
class_Acting or Performing machine           2
Name: count, dtype: int64.


In [20]:
class_list = class_names

In [21]:
trues = [class_list[int(y)] for _x, y in validation_ds.unbatch()]

### Print class counts for each split

In [22]:
df([y for y in trues]).value_counts()

class_Superior human                     22464
class_Thinking machine                       4
class_Acting or Performing machine           2
class_Collaborative or Interactive AI        2
class_Learning or recognition machine        2
class_Complex AI                             2
class_None of the above                      2
class_Mysterious AI                          2
Name: count, dtype: int64

Some of these counts are horrible. The number of examples in 'Learning or recognition machine', 'Superior human' and 'Mysterious AI' are really too small. Also, the 'None of the above' class is very large, although that's not really a problem.

### Load our trained model

In [23]:
import tensorflow as tf

# model = tf.keras.models.load_model('./fine_tuned_model_3m6herki')
# model = tf.keras.models.load_model('./fine_tuned_model_3f0vzk68')
# model = tf.keras.models.load_model('./fine_tuned_model_qdesgan9')
# model = tf.keras.models.load_model('./clip_features_model_kuw3ehqp')
model = tf.keras.models.load_model('./clip_features_model_kf5cnvvi')

### Make predictions

In [24]:
logits = model.predict(validation_ds)
predicted = [class_list[v] for v in np.argmax(logits, 1)]
from pandas import DataFrame as df

print(df(predicted).value_counts())
print(df(trues).value_counts())

1405/1405 [==============================] - 1s 622us/step
class_Complex AI                         8557
class_Mysterious AI                      4418
class_Acting or Performing machine       3849
class_Thinking machine                   2824
class_None of the above                  1737
class_Learning or recognition machine     828
class_Superior human                      267
Name: count, dtype: int64
class_Superior human                     22464
class_Thinking machine                       4
class_Acting or Performing machine           2
class_Collaborative or Interactive AI        2
class_Learning or recognition machine        2
class_Complex AI                             2
class_None of the above                      2
class_Mysterious AI                          2
Name: count, dtype: int64


In [25]:
df_predicted = df(predicted).reset_index()
df_trues = df(trues).reset_index()

In [26]:
df_predicted = df_predicted.rename(columns={0:'predictions'})
df_trues = df_trues.rename(columns={0:'folder_validation'})

In [27]:
results = []

In [28]:
for prediction, path in zip(predicted, paths):
    r = {}
    r['prediction'] = prediction
    r['path'] = path
    r['filename'] = path.name
    results.append(r)

In [29]:
results = pd.DataFrame(results)

In [30]:
results

,prediction,path,filename
0,class_Mysterious AI,/home/theo/robot_images/images_by_class/valida...,139716097.jpg
1,class_Mysterious AI,/home/theo/robot_images/images_by_class/valida...,139716088.jpg
2,class_Acting or Performing machine,/home/theo/robot_images/images_by_class/valida...,139674805.jpg
3,class_Learning or recognition machine,/home/theo/robot_images/images_by_class/valida...,139675342.jpg
4,class_None of the above,/home/theo/robot_images/images_by_class/valida...,139722757.jpg
...,...,...,...
22475,class_Thinking machine,/home/theo/robot_images/images_by_class/valida...,139689554.jpg
22476,class_Thinking machine,/home/theo/robot_images/images_by_class/valida...,139715503.jpg
22477,class_Thinking machine,/home/theo/robot_images/images_by_class/valida...,139689692.jpg
22478,class_Acting or Performing machine,/home/theo/robot_images/images_by_class/valida...,139722773.jpg


In [31]:
results.to_csv('predictions_imago.csv', index=False)
results.to_pickle('predictions_imago.pkl')
